# GENETIC ALGORITHM WITH NEH HEURISTIC
using niche genetic algorithm to support diversity and exploration of different aspects of the possible solution , adapting an hierarchical structure for encoding as well as adaptative mutation probability , and using initialization from neh heuristic

In [1]:
import numpy as np
import math
import time
import random
import itertools
import queue
import pandas as pd
from IPython.display import display, Markdown

In [2]:
matrix = np.array([
    [34, 20, 57, 47, 62, 40, 74, 94,  9, 62, 86, 13, 78, 46, 83, 52, 13, 70, 40, 60],
    [ 5, 48, 80, 43, 34,  2, 87, 68, 28, 84, 30, 35, 42, 39, 85, 34, 36,  9, 96, 84],
    [86, 35,  5, 93, 74, 12, 40, 95, 80,  6, 92, 14, 83, 49, 36, 38, 43, 89, 94, 33],
    [28, 39, 55, 21, 25, 88, 59, 40, 90, 18, 33, 10, 59, 92, 15, 77, 31, 85, 85, 99],
    [ 8, 91, 45, 55, 75, 18, 59, 86, 45, 89, 11, 54, 38, 41, 64, 98, 83, 36, 61, 19]
])
m=matrix.shape[0]
print(m)
n=matrix.shape[1]
print(n)
cost = np.transpose(matrix)
Npop=4

5
20


# Initialization using NEH heuristics

In [3]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)

    # Check if the sequence is empty
    if num_jobs == 0:
        # Return a default value (you may choose 0 or another suitable value)
        return 0

    completion_times = np.zeros((num_jobs, num_machines))

    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]

    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]

    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [4]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    """
    Orders jobs in descending order of their total processing times across all machines.

    Args:
    - processing_times: A 2D numpy array where each row represents a job and each column represents a machine.

    Returns:
    - A list of job indices sorted by their total processing times in descending order.
    """
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

def insertion(sequence, position, value):
    """
    Inserts a given value at a specified position in a sequence.

    """
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq
def neh_algorithm(processing_times):
    """
    Implements the NEH algorithm

    Args:
    - processing_times: A 2D numpy array where each row represents a job and each column represents a machine.

    Returns:
    - The sequence of jobs that minimizes the makespan and the corresponding makespan value.
    """
    # Order jobs by their total processing times in descending order
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)

    # Initialize the sequence with the first two jobs, choosing the order that minimizes the makespan
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]  # Remove the first two jobs from the ordered list

    # Iteratively add the remaining jobs to the sequence in the best position
    for job in ordered_sequence:
        Cmax = float('inf')  # Initialize the minimum makespan found so far
        best_sequence = []  # Initialize the best sequence found so far
        # Try inserting the current job in all possible positions and choose the one with the lowest makespan
        for i in range(len(sequence) + 1):
            new_sequence = insertion(sequence, i, job)  # Generate a new sequence with the current job inserted
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)  # Evaluate the makespan of the new sequence
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval  # Update the minimum makespan if a better one is found
                best_sequence = new_sequence  # Update the best sequence found so far
        sequence = best_sequence


         # Update the sequence to the best one found in this iteration

    # Compute the makespan of the final sequence
    Cmax = evaluate_sequence(sequence, processing_times)
    return sequence, Cmax

In [5]:
solution=neh_algorithm(cost)

In [6]:
import random

def generate_perturbed_solution(solution):
    """
    Generates a perturbed solution by randomly swapping adjacent jobs.
    """
    perturbed_solution = solution.copy()
    num_jobs = len(perturbed_solution)
    # Perform a random number of swaps (can be adjusted based on problem characteristics)
    num_swaps = random.randint(1, num_jobs // 2)  # Limit the number of swaps to prevent excessive perturbation
    for _ in range(num_swaps):
        # Randomly select two adjacent jobs to swap
        idx = random.randint(0, num_jobs - 2)  # Ensure that idx is within bounds
        perturbed_solution[idx], perturbed_solution[idx + 1] = perturbed_solution[idx + 1], perturbed_solution[idx]
    return perturbed_solution



In [8]:
def initialization(solution, processing_times, Npop):
    """
    Generates Npop solutions by perturbing the given solution.
    """
    solutions = [(solution, evaluate_sequence(solution, processing_times))]
    for _ in range(Npop - 1):
        perturbed_solution = generate_perturbed_solution(solution)
        makespan = evaluate_sequence(perturbed_solution, processing_times)
        solutions.append((perturbed_solution, makespan))
    # Sort solutions by makespan and keep only the top Npop solutions
    solutions.sort(key=lambda x: x[1])
    return solutions[:Npop]


In [9]:
solutions=initialization(solution[0], cost, Npop)
population = [solution[0] for solution in solutions]
print(population)

[[11, 16, 15, 1, 6, 12, 13, 18, 3, 2, 4, 5, 7, 19, 0, 9, 8, 14, 17, 10], [11, 16, 1, 15, 6, 12, 13, 18, 3, 7, 2, 5, 4, 19, 0, 9, 8, 14, 17, 10], [11, 15, 16, 1, 6, 12, 13, 3, 18, 2, 4, 7, 5, 0, 19, 9, 8, 17, 14, 10], [15, 11, 16, 1, 6, 12, 13, 18, 3, 5, 2, 4, 7, 0, 19, 9, 17, 8, 14, 10]]


# Calculate Makespane

In [10]:

def calculateObj(sol):
    qTime = queue.PriorityQueue()

    qMachines = []
    for i in range(m):
        qMachines.append(queue.Queue())

    for i in range(n):
        qMachines[0].put(sol[i])

    busyMachines = []
    for i in range(m):
        busyMachines.append(False)

    time = 0

    job = qMachines[0].get()
    qTime.put((time+cost[job][0], 0, job))
    busyMachines[0] = True

    while True:
        time, mach, job = qTime.get()
        if job == sol[n-1] and mach == m-1:
            break
        busyMachines[mach] = False
        if not qMachines[mach].empty():
                j = qMachines[mach].get()
                qTime.put((time+cost[j][mach], mach, j))
                busyMachines[mach] = True
        if mach < m-1:
            if busyMachines[mach+1] == False:
                qTime.put((time+cost[job][mach+1], mach+1, job))
                busyMachines[mach+1] = True
            else:
                qMachines[mach+1].put(job)
    return time


# Selection

In [25]:
def selection(pop):
    popObj = []
    for i in range(len(pop)):
        popObj.append([calculateObj(pop[i]), i])

    popObj.sort(reverse=True)

    distr = []
    distrInd = []
    for i in range(len(pop)):
        distrInd.append(popObj[i][1])
        prob = (2*(i+1)) / (len(pop) * (len(pop)+1))
        distr.append(prob)
    parents = []
    for i in range(len(pop)):
    # Randomly choose a pair of parents
     inter = list(np.random.choice(distrInd, 2, p=distr))

    # Check if the chosen pair already exists in the parents list
     if inter not in parents:
        # If not, append it to the parents list
         parents.append(inter)
     else:
        # If it already exists, regenerate the pair until a unique one is found
         while inter in parents:
             inter = list(np.random.choice(distrInd, 2, p=distr))
         parents.append(inter)


    return parents

In [12]:
parents=selection(population)

# Crossover

# C1 Operation

In [13]:
def c1_oper(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:1])
    child = list(parents[0])
    for i in range(pos[0], len(child)):
        child[i] = -1
    p = -1
    for i in range(pos[0], len(child)):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    return child

# LOX Operation

In [14]:
def lox_oper(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:2])
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    child = list(parents[0])

    for i in range(0, pos[0]):
        child[i] = -1
    for i in range(pos[1],len(child)):
        child[i]= -1
    p = -1
    for i in range(0, pos[0]):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    for i in range(pos[1], len(child)):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    return child

# POX Operation

In [15]:
def pox_oper(parents):
    m=int(n/2)
    pos = list(np.random.permutation(np.arange(n-1)+1)[:m])
    child = list(parents[0])
    for i in range(0,len(child)):

        if i not in pos :
            child[i]=-1
    p = -1
    for i in range(0, len(child)):
        if i not in pos :
            while True:
                p = p + 1
                if parents[1][p] not in child:
                    child[i] = parents[1][p]
                    break
    return child

In [16]:
child=[]
print(parents)
for p in parents:
    r = random.random()
    if r < 1:
        child.append(pox_oper([population[p[0]], population[p[1]]]))
        print(child)
    else:
        if r < 0.5:
            child.append(population[p[0]])
        else:
            child.append(population[p[1]])

[[1, 1], [0, 0], [0, 3], [0, 1]]
[[11, 16, 1, 15, 6, 12, 13, 18, 3, 7, 2, 5, 4, 19, 0, 9, 8, 14, 17, 10]]
[[11, 16, 1, 15, 6, 12, 13, 18, 3, 7, 2, 5, 4, 19, 0, 9, 8, 14, 17, 10], [11, 16, 15, 1, 6, 12, 13, 18, 3, 2, 4, 5, 7, 19, 0, 9, 8, 14, 17, 10]]
[[11, 16, 1, 15, 6, 12, 13, 18, 3, 7, 2, 5, 4, 19, 0, 9, 8, 14, 17, 10], [11, 16, 15, 1, 6, 12, 13, 18, 3, 2, 4, 5, 7, 19, 0, 9, 8, 14, 17, 10], [15, 11, 16, 1, 6, 12, 13, 18, 3, 2, 4, 5, 7, 19, 0, 9, 8, 14, 17, 10]]
[[11, 16, 1, 15, 6, 12, 13, 18, 3, 7, 2, 5, 4, 19, 0, 9, 8, 14, 17, 10], [11, 16, 15, 1, 6, 12, 13, 18, 3, 2, 4, 5, 7, 19, 0, 9, 8, 14, 17, 10], [15, 11, 16, 1, 6, 12, 13, 18, 3, 2, 4, 5, 7, 19, 0, 9, 8, 14, 17, 10], [11, 16, 1, 15, 6, 12, 13, 18, 3, 2, 7, 5, 4, 19, 0, 9, 8, 14, 17, 10]]


# Mutation

## SWAP

In [17]:
def swap(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    m=sol[pos[0]]
    sol[pos[0]]=sol[pos[1]]
    sol[pos[1]]=m

    return sol

### INV

In [18]:
def INV(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    i=pos[0]
    j=pos[1]
    while (j >= i):
        m=sol[i]
        sol[i]=sol[j]
        sol[j]=m
        i=i+1
        j=j-1

    return sol

### Decalage

In [19]:
def decalage(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t

    remJob = sol[pos[1]]

    for i in range(pos[1], pos[0], -1):
        sol[i] = sol[i-1]

    sol[pos[0]] = remJob

    return sol

In [ ]:
print(child[0])
print(decalage(child[0]))

[10, 7, 17, 19, 11, 16, 14, 8, 1, 9, 0, 6, 18, 12, 5, 3, 4, 2, 13, 15]
[9, 4]
[10, 7, 17, 19, 9, 11, 16, 14, 8, 1, 0, 6, 18, 12, 5, 3, 4, 2, 13, 15]


# UPDATES

In [21]:
def elitistUpdate(oldPop, newPop):
    bestSolInd = 0
    bestSol = calculateObj(oldPop[0])

    for i in range(1, len(oldPop)):
        tempObj = calculateObj(oldPop[i])
        if tempObj < bestSol:
            bestSol = tempObj
            bestSolInd = i

    rndInd = random.randint(0,len(newPop)-1)

    newPop[rndInd] = oldPop[bestSolInd]

    return newPop

In [20]:

def findBestSolution(pop):
    bestObj = calculateObj(pop[0])
    avgObj = bestObj
    bestInd = 0
    for i in range(1, len(pop)):
        tObj = calculateObj(pop[i])
        avgObj = avgObj + tObj
        if tObj < bestObj:
            bestObj = tObj
            bestInd = i

    return bestInd, bestObj, avgObj/len(pop)

In [23]:
import pickle

# Load the tables variable from the file
with open(r"C:\Users\km_ka\Downloads\tables.pkl", 'rb') as f:
    tables = pickle.load(f)

print("Tables variable has been loaded from 'tables.pkl'.")


Tables variable has been loaded from 'tables.pkl'.


In [34]:
import random
import numpy as np
import time

import random
import numpy as np


#1206        1170

# Define your mutation and crossover methods here
table = np.array(tables[8])
Npop=3
stopGeneration = 10000
upper = 1800
Pc = 1
Pm = 1

m=table.shape[0]
print(m)
n=table.shape[1]
print(n)
cost = np.transpose(table)
# Initialize an empty list to store the results
results = []

# Run the algorithm
t1 = time.perf_counter()
solution=neh_algorithm(cost)
population=initialization(solution[0], cost, Npop)
population = [solution[0] for solution in population]
objectives=[]
for p in population:
    objectives.append (evaluate_sequence(p,cost))

for i in range(stopGeneration):
    # Select parents
    parents = selection(population)
    childs = []
    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(lox_oper([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                    childs.append(population[p[0]])
            else:
                    childs.append(population[p[1]])

    # Apply mutation
    for c in childs:
            r = random.random()
            if r < Pm:
                c = decalage(c)

    # Update the population
    population = elitistUpdate(population, childs)
    objectives=[]
    for p in population:
        objectives.append (evaluate_sequence(p,cost))


# Stop Timer
t2 = time.perf_counter()

# Results Time
bestSol, bestObj, avgObj = findBestSolution(population)

print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)



20
10
Population:
[[1, 9, 3, 6, 8, 5, 2, 0, 7, 4], [0, 1, 4, 3, 6, 9, 8, 5, 2, 7], [1, 3, 6, 9, 8, 5, 4, 7, 2, 0]]

Solution:
[1, 3, 6, 9, 8, 5, 4, 7, 2, 0]

Objective Value:
1669

Average Objective Value of Population:
1822.67

CPU Time (s)
122.17


In [ ]:
deviation = ((1210 - 1206) / 1206) / 100
print(deviation)

3.316749585406302e-05


In [ ]:
import random
import numpy as np
import time
#1206        1170
# Set parameters
Npop = 3
stopGeneration = 10000
Pc = 1
Pm = 1
nich_radius = 0

# Initialize an empty list to store the results
results = []

# Run the algorithm
t1 = time.perf_counter()
solution = neh_algorithm(cost)
population = initialization(solution[0], cost, Npop)
population = [solution[0] for solution in population]
objectives = []

for p in population:
    objectives.append(evaluate_sequence(p, cost))

for i in range(stopGeneration):
    # Select parents
    parents = selection(population)
    childs = []

    # Randomly choose crossover and mutation operations
    crossover_op = random.choice([lox_oper, pox_oper, c1_oper])
    mutation_op = random.choice([swap, INV, decalage])

    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(crossover_op([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                childs.append(population[p[0]])
            else:
                childs.append(population[p[1]])

    # Apply mutation
    for c in childs:
        r = random.random()
        if r < Pm:
            c = mutation_op(c)

    # Update the population
    population = elitistUpdate(population, childs)
    objectives = []

    for p in population:
        objectives.append(evaluate_sequence(p, cost))

# Stop Timer
t2 = time.perf_counter()

# Results Time
bestSol, bestObj, avgObj = findBestSolution(population)

print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)


Population:
[[16, 1, 15, 8, 3, 13, 11, 9, 17, 18, 6, 12, 4, 19, 7, 2, 0, 14, 5, 10], [16, 8, 15, 1, 3, 13, 11, 9, 17, 18, 6, 12, 4, 19, 7, 2, 0, 14, 5, 10], [16, 0, 1, 8, 3, 13, 11, 9, 17, 18, 6, 12, 4, 19, 7, 2, 5, 14, 15, 10]]

Solution:
[16, 1, 15, 8, 3, 13, 11, 9, 17, 18, 6, 12, 4, 19, 7, 2, 0, 14, 5, 10]

Objective Value:
1206

Average Objective Value of Population:
1281.33

CPU Time (s)
64.87


In [ ]:
deviation = ((1206 - 1206) / 1206) / 100
print(deviation)

0.0


In [ ]:
import numpy as np
# [ 3125        2907]
data = """46 52 79 45 97 10 44 24 85 75 66 49 95 61 19 47 84 13 11 19 98  2 85 44  7 73 19 69 12 73 85 23 53 16 88  8 26 42 58 63  7  2 44 38 24 76 85 61 32 90
 61 87 51 25 73 93 28 90 94 59 64  2 16 35 53 40 81 26 85  4  4 10 63 96 55 71 66 94  7 15 11 99 37 50 56 69 22 56 67 63 96 74  4 42 40 30 93 36 25 87
  3  1 58 85 33 71 58 56 64 43 48 69 96 35 82 53 64 11 61 36 53 87 88 10 32 38 25 24 90  7 11 49  2 76 17 32 39  9 83 69 67 28 88 23 91 71  3 26 41 96
 51 24 21 57 69 51 50 51 21 19 63 91 11  6 31 63 36 39 57 47 56 65 59  4 10 12 62 43 49 54 87 29  2 18 75 39 77 69 15 78 68 37 22 41 92 67 24 87 91 31
 37 16 42 47 94 14 94 34 72 36 88 51 41 71 94 99 11 97 44 77 69 91 38 25 87  7 66 54 86 49  3 48 44 93 37 82 31 59 78 33 36  3 58 10 98  6 44 62 24 94
 79 93 68 75 37 44 34 39 76 62 74 28 78 43 98 83 91 27  6 82 60 44 43 76 99 66 11 35 52  8 40 62 25 24 30  1 73 27 16 91 33 11 99  2 60 90 36 62 15  3
 83 87 38 38 86 67 23 19 97 78 66 67  7 23 67  8 77 71 85 29 49  3 94 76 95 48  4 37 82 57 61  6 97  5 27 95 46 92 46 52  8 11  7 54 72 57 85 22 87 65
 22 29 99 25 98 55 80 82 33 68 47 74 26 61 95 55 11 42 72 14  8 98 90 36 75 69 26 24 55 98 86 30 92 94 66 47  3 41 41 47 89 28 39 80 47 57 74 38 59  5
 27 92 75 94 18 41 37 58 56 20  2 39 91 81 33 14 88 22 36 65 79 23 66  5 15 51  2 81 12 40 59 32 16 87 78 41 43 94  1 93 22 93 62 53 30 34 27 30 54 77
 24 47 39 66 41 46 24 23 68 50 93 22 64 81 94 97 54 82 11 91 23 32 26 22 12 23 34 87 59  2 38 84 62 10 11 93 57 81 10 40 62 49 90 34 11 81 51 21 39 27"""

# Parse the string data into a NumPy array
matrix = np.fromstring(data, sep=' ', dtype=int)
matrix = matrix.reshape(10, -1)  # Reshape the array into a 10x50 matrix
m=matrix.shape[0]
print(m)
n=matrix.shape[1]
print(n)
cost = np.transpose(matrix)

10
50


In [ ]:
import random
import numpy as np
import time

import random
import numpy as np




# Define your mutation and crossover methods here

# Set parameters
Npop = 3
stopGeneration = 10000
Pc = 1
Pm = 1
nich_radius=0
# Initialize an empty list to store the results
results = []

# Run the algorithm
t1 = time.perf_counter()
solution=neh_algorithm(cost)
population=initialization(solution[0], cost, Npop)
population = [solution[0] for solution in population]
objectives=[]
for p in population:
    objectives.append (evaluate_sequence(p,cost))

for i in range(stopGeneration):
    # Select parents
    parents = selection(population)
    childs = []
    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(lox_oper([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                    childs.append(population[p[0]])
            else:
                    childs.append(population[p[1]])

    # Apply mutation
    for c in childs:
            r = random.random()
            if r < Pm:
                c = decalage(c)

    # Update the population
    population = elitistUpdate(population, childs)
    objectives=[]
    for p in population:
        objectives.append (evaluate_sequence(p,cost))


# Stop Timer
t2 = time.perf_counter()

# Results Time
bestSol, bestObj, avgObj = findBestSolution(population)

print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)

Population:
[[24, 30, 17, 43, 19, 36, 33, 32, 41, 47, 46, 48, 45, 14, 13, 31, 9, 44, 39, 28, 25, 37, 34, 42, 35, 11, 5, 12, 0, 27, 2, 40, 8, 4, 20, 10, 26, 1, 21, 23, 3, 29, 16, 7, 6, 15, 49, 22, 18, 38], [24, 30, 17, 43, 19, 36, 33, 32, 41, 47, 46, 48, 45, 22, 14, 13, 31, 9, 44, 39, 28, 25, 37, 34, 42, 35, 11, 5, 12, 0, 27, 2, 40, 8, 4, 20, 10, 26, 1, 21, 23, 3, 29, 16, 7, 6, 15, 49, 18, 38], [24, 30, 17, 43, 44, 19, 36, 33, 32, 41, 47, 46, 48, 45, 14, 13, 37, 31, 9, 39, 28, 25, 2, 34, 42, 35, 11, 5, 12, 0, 27, 40, 8, 4, 20, 10, 26, 1, 21, 23, 3, 29, 16, 7, 6, 15, 49, 22, 18, 38]]

Solution:
[24, 30, 17, 43, 19, 36, 33, 32, 41, 47, 46, 48, 45, 14, 13, 31, 9, 44, 39, 28, 25, 37, 34, 42, 35, 11, 5, 12, 0, 27, 2, 40, 8, 4, 20, 10, 26, 1, 21, 23, 3, 29, 16, 7, 6, 15, 49, 22, 18, 38]

Objective Value:
3131

Average Objective Value of Population:
3174.67

CPU Time (s)
322.12


In [ ]:
deviation = ((3131 - 3125) / 3125) / 100
print(deviation)

1.92e-05


In [ ]:
import random
import numpy as np
import time
#1206        1170
# Set parameters
Npop = 3
stopGeneration = 10000
Pc = 1
Pm = 1
nich_radius = 0

# Initialize an empty list to store the results
results = []

# Run the algorithm
t1 = time.perf_counter()
solution = neh_algorithm(cost)
population = initialization(solution[0], cost, Npop)
population = [solution[0] for solution in population]
objectives = []

for p in population:
    objectives.append(evaluate_sequence(p, cost))

for i in range(stopGeneration):
    # Select parents
    parents = selection(population)
    childs = []

    # Randomly choose crossover and mutation operations
    crossover_op = random.choice([lox_oper, pox_oper, c1_oper])
    mutation_op = random.choice([swap, INV, decalage])

    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(crossover_op([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                childs.append(population[p[0]])
            else:
                childs.append(population[p[1]])

    # Apply mutation
    for c in childs:
        r = random.random()
        if r < Pm:
            c = mutation_op(c)

    # Update the population
    population = elitistUpdate(population, childs)
    objectives = []

    for p in population:
        objectives.append(evaluate_sequence(p, cost))

# Stop Timer
t2 = time.perf_counter()

# Results Time
bestSol, bestObj, avgObj = findBestSolution(population)

print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)


Population:
[[19, 48, 33, 30, 14, 2, 17, 36, 28, 41, 43, 9, 13, 11, 3, 24, 10, 5, 12, 46, 47, 32, 37, 31, 20, 40, 0, 34, 29, 42, 35, 23, 26, 39, 45, 8, 6, 1, 21, 44, 27, 25, 4, 16, 15, 49, 7, 22, 18, 38], [19, 48, 33, 30, 14, 2, 17, 36, 28, 41, 43, 9, 13, 11, 3, 24, 10, 5, 12, 46, 47, 32, 37, 31, 20, 8, 45, 39, 26, 23, 35, 42, 29, 34, 0, 40, 6, 1, 21, 44, 27, 25, 4, 16, 15, 49, 7, 22, 18, 38], [19, 29, 33, 30, 14, 2, 17, 36, 28, 41, 43, 9, 13, 11, 3, 24, 10, 5, 12, 46, 47, 32, 37, 31, 20, 40, 0, 34, 48, 23, 35, 6, 26, 39, 45, 8, 44, 1, 21, 42, 49, 15, 16, 4, 25, 27, 7, 22, 18, 38]]

Solution:
[19, 48, 33, 30, 14, 2, 17, 36, 28, 41, 43, 9, 13, 11, 3, 24, 10, 5, 12, 46, 47, 32, 37, 31, 20, 40, 0, 34, 29, 42, 35, 23, 26, 39, 45, 8, 6, 1, 21, 44, 27, 25, 4, 16, 15, 49, 7, 22, 18, 38]

Objective Value:
3119

Average Objective Value of Population:
3272.00

CPU Time (s)
308.58


In [ ]:
deviation = ((3119 - 3125) / 3125) / 100
print(deviation)


-1.92e-05


In [ ]:
import numpy as np

data = """ 52 95 42 75 44 57 89 53 84 62 91 14 95 89  4 95  2 97 68 20 33 51 98  8 85 86 73  4 40 98 12 59 44 46  2 41 28 83 28 21 80 71  4 60 34 55 53 96 37 37 63 99 69 70 53 21 10 31 80 18  5 18 17 71 90 93 14 49 52  7 78 57 41 75 98 93 33 75 68 33 60 82 24 99  4 97 24 50 55 91 46 58 17 47 82  6 15 91 74 42
 82 21 79 95 46 23 40 95 87 37 24 24 65 62 19 67 66  6 65 59  2 67 82 90 30 63  5 93 53 85 81 73 34 74 13 78 35 20 16 48 12 11 80  9 24 76 32 35 66 48 16 26 46 66 76 31 36  8 37 21  3 76 67  5 47 72 66 56 95 49 47 26 81 56 76 66 36 53 26 52 29 36 68 21 71 61 71 69 28 86 27 41 86 55 17 62 96 59 53 93
 63 55 59 35 21 59 78 25 30 38 78 79 58 44 38 76 70 72 85  8 10 84 42 67 20 24 75 23 33 60 20 75 83 26 92 29 39 14 74 66 86 10 27  8  7 97 84 56 61  9 94 34 89 62 47 66 76 15 18 54 24 55 96 10 12 96 53 92 77  6 91 14 41 30 85 17 23 60 76 39 85 10 65 15 55 41 28 93 88 27 77 81 19 76 55 67 65  8 18 56
 79 21 93 32  8 45 37 78 26 98 17 25 21 28 68 24 62 89 60 64 38 90 87  1 99 34  9 22 74 14 14 84 75 37 32 29 32 89 12 47 19 97  7 12 43 89 14 33 56 57 22  6 24 55 48 57 78  5 50 83 70 21 71 58 36 50 31 86 29 30 93 49 83 89 44 38 62 45 22 85 39 98 56 68 84 77 67 53 46 24 52 96  2 88 33 27 49 78 82 65
 80 13 64 77 17 78 82  4 72 93 68 25 67 80 43 93 21 33 14 30 59 83 85 85 70 35  2 76 46 72 69 46  3 57 71 77 33 49 59 82 59 70 76 10 65 19 77 86 21 75 96  3 50 57 66 84 98 55 70 32 31 64 11  9 32 58 98 95 25  4 45 60 87 31  1 96 22 95 73 77 30 88 14 22 93 48 10  7 14 91  5 43 30 79 39 34 77 81 11 10
 53 19 99 62 88 93 34 72 42 65 39 79  9 26 72 29 36 48 57 95 93 79 88 77 94 39 74 46 17 30 62 77 43 98 48 14 45 25 98 30 90 92 35 13 75 55 80 67  3 93 54 67 25 77 38 98 96 20 15 36 65 97 27 25 61 24 97 61 75 92 73 21 29  3 96 51 26 44 56 31 64 38 44 46 66 31 48 27 82 51 90 63 85 36 69 67 81 18 81 72
 71 90 59 82 22 88 35 49 78 69 76  2 14  3 22 26 44  1  4 16 55 43 87 35 76 98 78 81 48 25 81 27 84 59 98 14 32 95 30 13 68 19 57 65 13 63 26 96 53 94 27 93 49 63 65 34 10 56 51 97 52 46 16 50 96 85 61 76 30 90 42 88 37 43 88 91 14 63 65 74 71  8 39 95 82 17 38 69 17 24 66 75 52 59  4 73 56 19 39 51
 95 53 54 22 84 54  2 80 84 66 25 16 79 90 51 29 29 90 83 83 19 95 87 12 34 23 44 30 82 83 42 56 89 38 96 10  3 53 97 11 65 47 76 22 17 14 11 69 91 53  3 80 78 32 53 43 85 19 48 49 66 22 37 51 82 59 88 77 19 32 52  9 96 23 64 22 37  3 52 44 11 21 85  6 40 68 30 35 58 31 11 11  6 59 64 65 23 80 75 63
 92 62 11 83 87 66 98 42 23 45 52  6  3 64 55 97 83 42 81 92 68 46 56 88 50 13 23 13 49 18 50 94 71 64 31 21  2 63 58 36 64 52  8 94 51 36 82 30 17 21 80 38 55 34 85 44 47 66 19 66 61 60 98 82 79 71 28 74 27 33 13  9 12 51 16 49 83 48 13 78 96 77 68 88 77 76 73 92 72 87 66 98 40 31 75 45 98 90  4 23
 61 86 16 42 14 92 67 77 46 41 78  3 72 95 53 59 34 66 42 63 27 92  8 65 34  6 42 39  2  7 85 32 14 74 59 95 48 37 59  4 42 93 32 30 16 95 58 12 95 21 74 38  4 31 62 39 97 57  9 54 13 47  6 70 19 97 41  1 57 60 62 14 90 76 12 89 37 35 91 69 55 48 56 84 22 51 43 50 62 61 10 87 99 40 91 64 62 53 33 16"""

# Parse the string data into a NumPy array
matrix = np.fromstring(data, sep=' ', dtype=int)
matrix = matrix.reshape(10, -1)  # Reshape the array into a 10x50 matrix
m=matrix.shape[0]
print(m)
n=matrix.shape[1]
print(n)
cost = np.transpose(matrix)

10
100


In [ ]:
import random
import numpy as np
import time
#1206        1170
# Set parameters
Npop = 3
stopGeneration = 10000
Pc = 1
Pm = 1
nich_radius = 0

# Initialize an empty list to store the results
results = []

# Run the algorithm
t1 = time.perf_counter()
solution = neh_algorithm(cost)
population = initialization(solution[0], cost, Npop)
population = [solution[0] for solution in population]
objectives = []

for p in population:
    objectives.append(evaluate_sequence(p, cost))

for i in range(stopGeneration):
    # Select parents
    parents = selection(population)
    childs = []

    # Randomly choose crossover and mutation operations
    crossover_op = random.choice([lox_oper, pox_oper, c1_oper])
    mutation_op = random.choice([swap, INV, decalage])

    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(crossover_op([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                childs.append(population[p[0]])
            else:
                childs.append(population[p[1]])

    # Apply mutation
    for c in childs:
        r = random.random()
        if r < Pm:
            c = mutation_op(c)

    # Update the population
    population = elitistUpdate(population, childs)
    objectives = []

    for p in population:
        objectives.append(evaluate_sequence(p, cost))

# Stop Timer
t2 = time.perf_counter()

# Results Time
bestSol, bestObj, avgObj = findBestSolution(population)

print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)


Population:
[[69, 66, 60, 14, 96, 90, 87, 33, 75, 19, 79, 94, 26, 3, 46, 28, 59, 30, 40, 24, 92, 6, 57, 85, 9, 76, 32, 38, 36, 20, 80, 72, 93, 65, 84, 4, 63, 54, 78, 77, 18, 0, 95, 8, 35, 83, 31, 34, 41, 39, 99, 97, 43, 10, 7, 98, 23, 5, 71, 81, 82, 56, 1, 86, 22, 55, 17, 42, 16, 73, 52, 51, 50, 48, 27, 61, 89, 88, 47, 37, 68, 21, 15, 2, 64, 45, 12, 13, 91, 49, 70, 25, 62, 29, 44, 67, 74, 53, 58, 11], [69, 60, 14, 96, 90, 87, 33, 75, 19, 79, 94, 26, 3, 46, 28, 59, 30, 40, 24, 92, 6, 57, 85, 9, 76, 32, 38, 36, 20, 80, 72, 93, 65, 66, 84, 4, 63, 54, 78, 77, 18, 0, 95, 8, 35, 83, 31, 34, 41, 39, 99, 97, 43, 10, 7, 98, 23, 5, 71, 81, 82, 56, 1, 86, 22, 55, 17, 42, 16, 73, 52, 51, 50, 48, 27, 61, 89, 88, 47, 37, 68, 21, 15, 2, 64, 45, 12, 13, 91, 49, 70, 25, 62, 29, 44, 67, 74, 53, 58, 11], [69, 60, 14, 96, 57, 87, 24, 28, 46, 19, 79, 94, 29, 3, 26, 75, 59, 30, 40, 92, 33, 90, 6, 9, 85, 76, 32, 38, 36, 20, 93, 80, 72, 65, 66, 84, 4, 8, 63, 54, 78, 77, 18, 0, 95, 83, 35, 31, 34, 41, 39, 99, 

In [ ]:
deviation = ((avgObj - 5770 ) / 5770 ) / 100
print(deviation)

0.0004552281917966498
